In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Data Preparation
Adjusting note representation to include note duration and time offsets.

In [ ]:
import os
import numpy as np
from music21 import converter, instrument, note, chord

def read_midi(file):
    print("Loading Music File:", file)
    notes = []

    midi = converter.parse(file)
    parts = instrument.partitionByInstrument(midi)
    relevant_parts = parts.parts if parts else [midi]

    for part in relevant_parts:
        if 'Violin' in str(part.getInstrument()) or 'Violin' in str(part.partName):
            for element in part.recurse():
                if isinstance(element, note.Note):
                    notes.append((str(element.pitch), element.duration.quarterLength, element.offset))
                elif isinstance(element, chord.Chord):
                    notes.append(('.'.join(str(n) for n in element.normalOrder), element.duration.quarterLength, element.offset))
                elif isinstance(element, note.Rest):
                    notes.append(('rest', element.duration.quarterLength, element.offset))

    return notes


path = '/content/gdrive/MyDrive/Violin_Comp_Data/all_midi_files/'
files = [i for i in os.listdir(path) if i.endswith(".mid")]
notes_array = [read_midi(os.path.join(path, file)) for file in files]


### Encoding each unique note to an integer

In [3]:
# Flatten
all_notes = [note for sequence in notes_array for note in sequence]

# Mapping from notes to integers
note_to_int = {note: i for i, note in enumerate(sorted(set(all_notes)))}

# Encode sequences
input_sequences = []
output_notes = []
no_of_timesteps = 32

for notes in notes_array:
    for i in range(len(notes) - no_of_timesteps):
        input_seq = notes[i:i + no_of_timesteps]
        output_note = notes[i + no_of_timesteps]
        input_sequences.append([note_to_int[note] for note in input_seq])
        output_notes.append(note_to_int[output_note])

x_seq = np.array(input_sequences)
y_seq = np.array(output_notes)


### Initiating Train-Test Split & Reshaping Input for LSTM Model

In [4]:
from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(x_seq, y_seq, test_size=0.2, random_state=13)
x_tr = np.reshape(x_tr, (x_tr.shape[0], no_of_timesteps, 1))
x_val = np.reshape(x_val, (x_val.shape[0], no_of_timesteps, 1))


### LSTM Model Architecture
Trying a LSTM-based architecture for better sequence generation.

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(256, input_shape=(no_of_timesteps, 1), return_sequences=True))
model.add(LSTM(256))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(note_to_int), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32, 256)           264192    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4915)              1263155   
                                                                 
Total params: 2118451 (8.08 MB)
Trainable params: 2118451 (8.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Model Checkpoint

In [6]:
from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model_lstm1.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

### Train Model

In [7]:
history = model.fit(x_tr, y_tr, epochs=50, batch_size=128, validation_data=(x_val, y_val), callbacks=[mc])

Epoch 1/50
13/13 [==============================] - ETA: 0s - loss: 8.5146 - accuracy: 6.1425e-04
Epoch 1: val_loss improved from inf to 8.51159, saving model to best_model_violin.h5
13/13 [==============================] - 14s 772ms/step - loss: 8.5146 - accuracy: 6.1425e-04 - val_loss: 8.5116 - val_accuracy: 0.0000e+00
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - ETA: 0s - loss: 8.1360 - accuracy: 0.0043
Epoch 2: val_loss did not improve from 8.51159
13/13 [==============================] - 7s 513ms/step - loss: 8.1360 - accuracy: 0.0043 - val_loss: 9.5193 - val_accuracy: 0.0000e+00
Epoch 3/50
13/13 [==============================] - ETA: 0s - loss: 7.6154 - accuracy: 0.0037
Epoch 3: val_loss did not improve from 8.51159
13/13 [==============================] - 9s 688ms/step - loss: 7.6154 - accuracy: 0.0037 - val_loss: 9.8468 - val_accuracy: 0.0000e+00
Epoch 4/50
13/13 [==============================] - ETA: 0s - loss: 7.4340 - accuracy: 0.0037
Epoch 4: val_loss did not improve from 8.51159
13/13 [==============================] - 7s 514ms/step - loss: 7.4340 - accuracy: 0.0037 - val_loss: 10.4366 - val_accuracy: 0.0000e+00
Epoch 5/50
13/13 [==============================] - ETA: 0s - loss: 7.3774 - accuracy: 0.0049
Epoch 5: val_loss did not improve from 8.51159
13/13 [==============================] - 

### Load Best Model

In [8]:
from keras.models import load_model
model = load_model('best_model_lstm1.h5')

### Generate Music Predictions

In [12]:
def generate_music(model, start_sequence, length=50):
    prediction_output = []

    # Ensure start_sequence is correctly formatted
    start_sequence_formatted = np.array([note_to_int[note] for note in start_sequence])

    for note_index in range(length):
        prediction_input = np.reshape(start_sequence_formatted, (1, len(start_sequence_formatted), 1))
        prob = model.predict(prediction_input)[0]
        index = np.random.choice(range(len(prob)), p=prob)
        predicted_note = x_int_to_note[index]  # Use the inverse mapping
        prediction_output.append(predicted_note)

        # Update start_sequence_formatted for the next prediction
        start_sequence_formatted = np.append(start_sequence_formatted, [index])[1:]

    return prediction_output

# Create the inverse mapping from integers back to note tuples
x_int_to_note = dict((number, note) for note, number in note_to_int.items())

### Convert Back to MIDI

In [15]:
from music21 import stream, instrument, note, chord, midi

def midi_number_to_note_name(midi_number):
    # Converts a MIDI note number to a note name
    return midi.translate.pitchToNoteName(midi_number)

def convert_to_midi(prediction_output):
    midi_stream = stream.Stream()
    midi_stream.append(instrument.Violin())

    offset = 0
    for note_info in prediction_output:
        note_name = note_info[0]

        # Convert MIDI note numbers to note names
        if note_name.isdigit():
            note_name = midi_number_to_note_name(int(note_name))

        # Create note or rest
        if note_name != 'rest':
            new_note = note.Note(note_name)
        else:
            new_note = note.Rest()

        new_note.duration.quarterLength = note_info[1]
        new_note.offset = offset
        new_note.storedInstrument = instrument.Violin()
        midi_stream.append(new_note)
        offset += note_info[2]

    midi_stream.write('midi', fp='lstm_music1.mid')

In [ ]:
# Randomly select a starting sequence from x_val
random_index = np.random.randint(0, len(x_val))
start_sequence = x_val[random_index]

# Since start_sequence is currently encoded as integers, decode it back to note information
start_sequence_decoded = [x_int_to_note[note] for note in start_sequence.flatten()]

# Generate music based on the starting sequence
prediction_output = generate_music(model, start_sequence_decoded)
convert_to_midi(prediction_output)
